**Importing Necessary Libraries**

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

**Mounting Google Drive - I have copied the training and test data in my google drive folder**

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Set the Path to Data**

In [21]:
import os
data_path = '/content/drive/MyDrive/AIP Assignment 1/Avinash Barak/Question 2/train'


**Define a Function to Load a Batch**

In [22]:
import pickle
def load_cifar10_batch(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict[b'data'], dict[b'labels']

**Load All Batches and Preprocess**

In [23]:
import numpy as np

# Initialize variables to hold the training data and labels
train_data = []
train_labels = []

# Load each data batch
for i in range(1, 4):  # Adjust the range if you have more batch files
    file_path = os.path.join(data_path, f'data_batch_{i}')
    data_batch, labels_batch = load_cifar10_batch(file_path)
    train_data.append(data_batch)
    train_labels.append(labels_batch)

# Convert to numpy arrays and concatenate batches
train_data = np.concatenate(train_data)
train_labels = np.concatenate(train_labels)

# Reshape the data to the format (num_samples, height, width, channels)
train_images = train_data.reshape((len(train_data), 3, 32, 32)).transpose(0, 2, 3, 1)

# Normalize pixel values to be between 0 and 1
train_images = train_images / 255.0

# Convert labels to numpy array
train_labels = np.array(train_labels)

**Load Test Data**

In [25]:
test_data_path = '/content/drive/MyDrive/AIP Assignment 1/Avinash Barak/Question 2/test'

In [27]:
# Load the test data batch
test_batch_file = os.path.join(test_data_path, 'test_batch')
test_data, test_labels = load_cifar10_batch(test_batch_file)

In [28]:
# Reshape and normalize the test data
test_images = test_data.reshape((len(test_data), 3, 32, 32)).transpose(0, 2, 3, 1)
test_images = test_images / 255.0

# Convert labels to numpy array
test_labels = np.array(test_labels)

**Define the architecture of Convolutional Neural Network**

In [31]:
model = models.Sequential()
# Convolutional layer with Sigmoid activation
model.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=(32, 32, 3)))
# Pooling layer
model.add(layers.MaxPooling2D((2, 2)))
# Convolutional layer with Sigmoid activation
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
# Pooling layer
model.add(layers.MaxPooling2D((2, 2)))
# Convolutional layer with Sigmoid activation
model.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))

# Flatten the output of the conv layers to feed into the dense layers
model.add(layers.Flatten())
# Dense layer with Sigmoid activation
model.add(layers.Dense(64, activation='sigmoid'))
# Output layer with Sigmoid activation and 10 units for CIFAR-10 classes
model.add(layers.Dense(10, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Summary of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_1 (Flatten)         (None, 1024)             

**Training it using the training data**

In [32]:
history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10
938/938 [==============================] - 7s 6ms/step - loss: 2.2160 - accuracy: 0.1584 - val_loss: 1.9870 - val_accuracy: 0.2984
Epoch 2/10
938/938 [==============================] - 4s 5ms/step - loss: 1.8559 - accuracy: 0.3320 - val_loss: 1.7595 - val_accuracy: 0.3621
Epoch 3/10
938/938 [==============================] - 5s 5ms/step - loss: 1.6624 - accuracy: 0.4022 - val_loss: 1.5582 - val_accuracy: 0.4362
Epoch 4/10
938/938 [==============================] - 5s 5ms/step - loss: 1.5384 - accuracy: 0.4434 - val_loss: 1.5333 - val_accuracy: 0.4426
Epoch 5/10
938/938 [==============================] - 4s 5ms/step - loss: 1.4720 - accuracy: 0.4685 - val_loss: 1.4437 - val_accuracy: 0.4739
Epoch 6/10
938/938 [==============================] - 6s 6ms/step - loss: 1.4153 - accuracy: 0.4879 - val_loss: 1.4207 - val_accuracy: 0.4863
Epoch 7/10
938/938 [==============================] - 4s 5ms/step - loss: 1.3688 - accuracy: 0.5053 - val_loss: 1.4035 - val_accuracy: 0.4899
Epoch 

**Testing performance**

In [33]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy with Sigmoid activation:', test_acc)


313/313 - 1s - loss: 1.3065 - accuracy: 0.5277 - 645ms/epoch - 2ms/step

Test accuracy with Sigmoid activation: 0.5277000069618225


**Defining a new model with the same architecture as before, but replacing the 'sigmoid' activations with 'relu'.**

In [34]:
model_relu = models.Sequential()
# Convolutional layer with ReLU activation
model_relu.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
# Pooling layer
model_relu.add(layers.MaxPooling2D((2, 2)))
# Convolutional layer with ReLU activation
model_relu.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Pooling layer
model_relu.add(layers.MaxPooling2D((2, 2)))
# Convolutional layer with ReLU activation
model_relu.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten the output of the conv layers to feed into the dense layers
model_relu.add(layers.Flatten())
# Dense layer with ReLU activation
model_relu.add(layers.Dense(64, activation='relu'))
# Output layer with 10 units for CIFAR-10 classes
model_relu.add(layers.Dense(10))

# Compile the model with the same optimizer and loss function as before
model_relu.compile(optimizer='adam',
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   metrics=['accuracy'])

# Summary of the model
model_relu.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 15, 15, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_7 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 4, 4, 64)          36928     
                                                                 
 flatten_2 (Flatten)         (None, 1024)             

**Now, training the model with ReLU activations using the training data**

In [35]:
history_relu = model_relu.fit(train_images, train_labels, epochs=10,
                              validation_data=(test_images, test_labels))

Epoch 1/10
938/938 [==============================] - 6s 5ms/step - loss: 1.6337 - accuracy: 0.3987 - val_loss: 1.3677 - val_accuracy: 0.5087
Epoch 2/10
938/938 [==============================] - 5s 5ms/step - loss: 1.2991 - accuracy: 0.5330 - val_loss: 1.1955 - val_accuracy: 0.5797
Epoch 3/10
938/938 [==============================] - 4s 4ms/step - loss: 1.1531 - accuracy: 0.5924 - val_loss: 1.1318 - val_accuracy: 0.5982
Epoch 4/10
938/938 [==============================] - 4s 5ms/step - loss: 1.0530 - accuracy: 0.6308 - val_loss: 1.1086 - val_accuracy: 0.6098
Epoch 5/10
938/938 [==============================] - 5s 5ms/step - loss: 0.9626 - accuracy: 0.6594 - val_loss: 1.0922 - val_accuracy: 0.6121
Epoch 6/10
938/938 [==============================] - 4s 5ms/step - loss: 0.8943 - accuracy: 0.6875 - val_loss: 1.0190 - val_accuracy: 0.6429
Epoch 7/10
938/938 [==============================] - 5s 5ms/step - loss: 0.8421 - accuracy: 0.7005 - val_loss: 1.0000 - val_accuracy: 0.6553
Epoch 

**Evaluating its performance on the test data**

In [36]:
test_loss_relu, test_acc_relu = model_relu.evaluate(test_images, test_labels, verbose=2)
print('\nTest accuracy with ReLU activation:', test_acc_relu)

313/313 - 1s - loss: 0.9864 - accuracy: 0.6723 - 632ms/epoch - 2ms/step

Test accuracy with ReLU activation: 0.6722999811172485


**Summary**

*   Test accuracy with Sigmoid activation: 0.5277000069618225
*   Test accuracy with ReLU activation: 0.6722999811172485





**Loading the Additional Test Data and Labels**

In [38]:
import numpy as np

# Replace these paths with the correct paths to your .npy files
additional_test_data_path = '/content/drive/MyDrive/AIP Assignment 1/Avinash Barak/Question 2/test_sets/test_additional.npy'
additional_test_labels_path = '/content/drive/MyDrive/AIP Assignment 1/Avinash Barak/Question 2/test_sets/labels.npy'

# Load the data and labels
additional_test_images = np.load(additional_test_data_path)
additional_test_labels = np.load(additional_test_labels_path)

# Assuming the additional test data might not be normalized
additional_test_images = additional_test_images / 255.0


**Predicting Using Trained Models**

In [39]:
# Predict using the Sigmoid model
predictions_sigmoid = model.predict(additional_test_images)
predictions_sigmoid_labels = np.argmax(predictions_sigmoid, axis=1)

# Predict using the ReLU model
predictions_relu = model_relu.predict(additional_test_images)
predictions_relu_labels = np.argmax(predictions_relu, axis=1)

313/313 [==============================] - 1s 2ms/step


**Evaluating the Accuracy**

In [40]:
print(predictions_sigmoid_labels.shape, predictions_sigmoid_labels.dtype)
print(predictions_relu_labels.shape, predictions_relu_labels.dtype)
print(additional_test_labels.shape, additional_test_labels.dtype)

(10000,) int64
(10000,) int64
(50000,) uint8


In [41]:
# Ensure the additional test labels are one-dimensional
additional_test_labels = additional_test_labels.squeeze()

In [42]:
print("Shape of sigmoid predictions:", predictions_sigmoid_labels.shape)
print("Shape of ReLU predictions:", predictions_relu_labels.shape)
print("Shape of additional test labels:", additional_test_labels.shape)
print("Unique values in additional test labels:", np.unique(additional_test_labels))


Shape of sigmoid predictions: (10000,)
Shape of ReLU predictions: (10000,)
Shape of additional test labels: (50000,)
Unique values in additional test labels: [0 1 2 3 4 5 6 7 8 9]


In [44]:
correct_additional_test_labels = additional_test_labels[:10000]


In [45]:
# Recalculate the accuracy for the Sigmoid model
accuracy_sigmoid = np.mean(predictions_sigmoid_labels == correct_additional_test_labels)

# Recalculate the accuracy for the ReLU model
accuracy_relu = np.mean(predictions_relu_labels == correct_additional_test_labels)

print(f'Accuracy on additional test set with Sigmoid activation: {accuracy_sigmoid * 100:.2f}%')
print(f'Accuracy on additional test set with ReLU activation: {accuracy_relu * 100:.2f}%')

Accuracy on additional test set with Sigmoid activation: 52.34%
Accuracy on additional test set with ReLU activation: 58.72%
